# This notebook generates the data defining the case study

A .csv file is created containing:
 - Origin (latitude; longitude; altitude)
 - Destination (latitude; longitude; altitude)
 - Time of departure (formatted; only in seconds)
 - Drone model
 - Purpose
 - Time of planning

For each drone forming the background traffic and surveillance drones traffic.

In [1]:
"""
We define a default path where the modules developed in USEPE are stored
"""

default_path = r"/home/ror/ws/bluesky"

"""
We define a second default path where the scenarios are stored
"""

default_path2 = r"/home/ror/ws/scenarios/scenario/USEPE/exercise_3"

In [2]:
import os
import sys
import configparser
import pickle
import osmnx as ox
import random
import pandas as pd
import datetime

os.chdir(default_path)
from usepe.city_model.utils import read_my_graphml, layersDict
from usepe.city_model.multi_di_graph_3D import MultiDiGrpah3D
from usepe.city_model.city_graph import cityGraph
from usepe.city_model.scenario_definition import createDeliveryCSV, createSurveillanceCSV, createBackgroundTrafficCSV, createScenarioCSV


# Configuration file

We create a config file with the following fields:

[City]

mode = rectangle

hannover_lat = 52.376

hannover_lon = 9.76

zone_size = 1000

hannover_lat_min = 52.412274

hannover_lat_max = 52.466148

hannover_lon_min = 9.638119

hannover_lon_max = 9.745655

import = True

imported_graph_path = data/exe_3_ref.graphml

[Layers]

number_of_layers = 9

layer_width = 25

[Corridors]

corridors = 

altitude = 250

delta_z = 25

speed = 50

acceleration_length = 50

file_path_corridors = usepe/city_model/data/usepe-hannover-corridors.geojson

[Segments]

import = True

path = data/reference_segments.pkl

[BuildingData]

lat_min = 52.412274

lat_max = 52.466148

lon_min = 9.638119

lon_max = 9.745655

divisions = 20

directory_hannover = /home/ror/ws/bluesky/usepe/city_model/data/hannover_building_data

[Options]

one_way = False

simplify = True

simplification_distance = 0

simplification_error = 2

[Outputs]

graph_path = data/exe_3_ref.graphml

[Strategic_Deconfliction]

ratio = 3

delay = 60


In [3]:
 # -------------- 1. CONFIGURATION FILE -----------------
"""
This section reads the configuration file.
Change the config_path to read the desired file
"""
# CONFIG
config_path = default_path2 + r"/settings_exe_3_ref.cfg"
config = configparser.ConfigParser()
config.read( config_path )

['/home/ror/ws/scenarios/scenario/USEPE/exercise_3/settings_exe_3_ref.cfg']

In [4]:
# -------------- 2. CITY GRAPH -------------------------
"""
This section creates a city graph or loads the graph defined in the 'City' section of the
configuration file.
"""
# City
os.chdir(default_path2)
if config['City'].getboolean( 'import' ):
    filepath = config['City']['imported_graph_path']
    G = read_my_graphml( filepath )
    G = MultiDiGrpah3D( G )
    # fig, ax = ox.plot_graph( G )
else:
    print('Check the path to the graph file')
    G = cityGraph( config )

Reading the graph...


In [5]:
# -------------- 3. SEGMENTS ----------------------------
"""
This section creates an airspace segmentation or loads the segmentation defined in the 'Segments'
section of the configuration file.
Comment it to neglect the segmentation
"""
os.chdir(default_path2)
if config['Segments'].getboolean( 'import' ):
    path = config['Segments']['path']
    with open( path, 'rb' ) as f:
        segments = pickle.load( f )
else:
    segments = None

# Data for the experiments

The densities to be tested in the experiments are
 - Low: 0.0964 simultaneous flights / km^2
 - Medium: 0.964 simultaneous flights / km^2
 - High: 9.64 simultaneous flights / km^2
 
The average flight duration calculated is: 1000 s

The simulation time considered is: 1h (3600 s)


In [6]:
# General
simulation_time = 3600

# Background Traffic

In [7]:
# Background traffic
density = {'Low': 0.0964, 'Medium': 0.964, 'High': 9.64}
avg_flight_duration = 1000

# Add background traffic
print('Creating background traffic...')
createBackgroundTrafficCSV(density['Medium'], avg_flight_duration, simulation_time, G, segments, config)

Creating background traffic...
Creating flight 1...
Creating flight 2...
Creating flight 3...
Creating flight 4...
Creating flight 5...
Creating flight 6...
Creating flight 7...
Creating flight 8...
Creating flight 9...
Creating flight 10...
Creating flight 11...
Creating flight 12...
Creating flight 13...
Creating flight 14...
Creating flight 15...
Creating flight 16...
Creating flight 17...
Creating flight 18...
Creating flight 19...
Creating flight 20...
Creating flight 21...
Creating flight 22...
Creating flight 23...
Creating flight 24...
Creating flight 25...
Creating flight 26...
Creating flight 27...
Point in no fly zone: lat 52.42776314109385, lon 9.744950632894497
Creating flight 28...
Creating flight 29...
Creating flight 30...
Creating flight 31...
Creating flight 32...
Creating flight 33...
Creating flight 34...
Creating flight 35...
Creating flight 36...
Creating flight 37...
Creating flight 38...
Creating flight 39...
Creating flight 40...
Creating flight 41...
Creating 

# Surveillance operation

In [8]:
origins = [(52.44893, 9.72996, 5),
           (52.43906, 9.69446, 5),
           (52.42035, 9.65075, 51),
           (52.42035, 9.65075, 51),
           (52.43218, 9.68395, 5)]

destinations = [(52.45044, 9.71846, 50),
                (52.44066, 9.68234, 50),
                (52.44252, 9.67161, 50),
                (52.43126, 9.67879, 50),
                (52.43572, 9.68334, 50)]

departure_times = [600, 600, 600, 660, 855]

drone_models = ['M600', 'M600', 'M600', 'M600', 'W178']

operation_ids = ['op1', 'op2', 'op3', 'op4', 'op5']

operation_durations = [600, 450, 330, 240, 660]

# Add operation scenario
print('Creating surveillance drones...')
createSurveillanceCSV(origins, destinations, departure_times, drone_models, operation_ids, operation_durations, simulation_time)

Creating surveillance drones...
Creating flight plan with origin: (52.44893, 9.72996) and destination: (52.45044, 9.71846)
Creating flight plan with origin: (52.43906, 9.69446) and destination: (52.44066, 9.68234)
Creating flight plan with origin: (52.42035, 9.65075) and destination: (52.44252, 9.67161)
Creating flight plan with origin: (52.42035, 9.65075) and destination: (52.43126, 9.67879)
Creating flight plan with origin: (52.43218, 9.68395) and destination: (52.43572, 9.68334)
Delivery drones stored in: /home/ror/ws/scenarios/scenario/USEPE/exercise_3/data/surveillance_600-600-600-660-855_3600.csv
